# k3sx

Install k3s on given ssh remotes via k3sup and setup additional components for a proper production cluster.

## Prerequesites

To proceed, you need one or more exposed remote Linux hosts, which you can access via public IPv4 and can login to as a privileged user via a private key.

## Setup

### 1. Supply all inputs

#### Set environment variables for SSH

In [ ]:
# user name and ssh key for login on your remotes
%env SSH_USER=root
%env SSH_KEY=~/.ssh/id_rsa

#### Create txt files and fill them with the IPv4 addresses of your remotes

One IP per line

In [ ]:
%%bash

touch ./server-ips.txt
touch ./agent-ips.txt

if command -v code &> /dev/null
then 
    code ./server-ips.txt
    code ./agent-ips.txt
fi

#### Copy values.yaml template to root and fill in all parameters

In [ ]:
%%bash

if [ ! -f ./values.yaml ]; 
then
    cp ./chart/values.yaml ./values.yaml
fi

if command -v code &> /dev/null
then 
    code ./values.yaml
fi

### 2. Install CLI tools

#### Quick install via Homebrew

Install [brew](https://brew.sh/) if you don't have it yet: 

```bash 
/bin/bash -c "$(curl -fsSL https://raw.githubusercontent.com/Homebrew/install/HEAD/install.sh)"
``` 

This requires sudo and must be done outside of this notebook.

In [ ]:
%%bash
brew install kubernetes-cli int128/kubelogin/kubelogin helm k3sup fluxcd/tap/flux vcluster cilium-cli

### 3. Setup k3s on all given ssh remotes

In [ ]:
%%bash

helm template k3sx ./charts/k3sx --values ./values.yaml \
    --set k3s_setup_run=true --output-dir ./rendered

In [ ]:
%%bash

for SERVER in $(cat ./server-ips.txt)
do
    ssh-keygen -f "$HOME/.ssh/known_hosts" -R "$SERVER"
    scp -o StrictHostKeyChecking=no -i $SSH_KEY ./rendered/k3sx/templates/k3s-config.yaml $SSH_USER@$SERVER:~/
    scp -i $SSH_KEY ./90-override.conf $SSH_USER@$SERVER:/etc/sysctl.d/ # Ubuntu 21+ quirk with Cilium
    ssh -i $SSH_KEY $SSH_USER@$SERVER "echo 'PubkeyAcceptedKeyTypes=+ssh-rsa' >> /etc/ssh/sshd_config & reboot" # Ubuntu 21+ quirk with k3sup
    sleep 30
    k3sup install --ip $SERVER --user $SSH_USER --ssh-key $SSH_KEY \
        --k3s-extra-args '--config ~/k3s-config.yaml --node-taint node.cilium.io/agent-not-ready=true:NoExecute'
    export JOIN_SERVER=$SERVER
done

In [ ]:
%%bash
for AGENT in $(cat ./agent-ips.txt)
do
    ssh-keygen -f "$HOME/.ssh/known_hosts" -R $AGENT
    scp -o StrictHostKeyChecking=no -i $SSH_KEY ./rendered/k3sx/templates/k3s-config.yaml $SSH_USER@$AGENT:~/
    scp -i $SSH_KEY ./90-override.conf $SSH_USER@$AGENT:/etc/sysctl.d/ # Ubuntu 21+ quirk with Cilium
    ssh -i $SSH_KEY $SSH_USER@$AGENT "echo 'PubkeyAcceptedKeyTypes=+ssh-rsa' >> /etc/ssh/sshd_config & reboot" # Ubuntu 21+ quirk with k3sup
    sleep 30
    k3sup join --ip $AGENT --server-ip 49.12.239.191 --user $SSH_USER --ssh-key $SSH_KEY \
        --k3s-extra-args '--node-taint node.cilium.io/agent-not-ready=true:NoExecute'
done

In [ ]:
%env KUBECONFIG=./kubeconfig

### 4. Install Cilium via Helm

Cilium must be deployed even before FluxCD as no pods can be started without it. This manual deployment will be reconciled with a GitOps deployment later.

In [ ]:
%%bash
helm repo add cilium https://helm.cilium.io/
helm install cilium cilium/cilium --version 1.11.4 \
   --namespace kube-system -f cilium-values.yaml 
sleep 30 # wait for cilium to be ready

In [ ]:
%%bash
cilium status

In [ ]:
%%bash
kubectl get node -o wide

### 5. [Bootstrap FluxCD](https://fluxcd.io/docs/installation/#bootstrap) 

In [ ]:
%%bash
flux install

#### Alternative

If you wish to have the entire cluster be controlled my a single GitOps repo, instead do [`flux bootstrap github`](https://fluxcd.io/docs/installation/#github-and-github-enterprise) for automatically setting up a GitOps repo on GitHub or [`flux bootstrap git`](https://fluxcd.io/docs/installation/#generic-git-server) for using an existing GitOps repo with an arbitrary provider.

### 6. Install the infrastructure chart

**NOTE:** If you chose to setup a GitOps repo for the entire cluster in the previous step, create and commit respective YAML definitions of [GitRepository](https://fluxcd.io/docs/guides/helmreleases/#git-repository) and [HelmRelease](https://fluxcd.io/docs/guides/helmreleases/#define-a-helm-release) resources to your repo instead of using the CLI commands shown below.

#### Create a GitRepository source from this repo

In [ ]:
%%bash

flux create source git k3sx-root \
  --url=https://github.com/lorenzo-w/k3sx \
  --branch=testing/justcilium

#### Create a HelmRelease with values from ./values.yaml

In [ ]:
%%bash

flux create hr k3sx \
  --source=GitRepository/k3sx-root \
  --chart=./charts/k3sx \
  --values=./values.yaml \
sleep 60 # wait for all sub-releases to be reconciled

#### Confirm that all HelmRelease reconciliations are done and successful

Otherwise wait a couple more seconds and refresh.

In [ ]:
%%bash
flux get hr --all-namespaces

And also confirm that no errors have occured

In [ ]:
%%bash
flux logs --level error

### 7. [Change the default StorageClass](https://kubernetes.io/docs/tasks/administer-cluster/change-default-storage-class/) to Longhorn

In [ ]:
%%bash

kubectl patch storageclass local-path -p '{"metadata": {"annotations":{"storageclass.kubernetes.io/is-default-class":"false"}}}'
kubectl patch storageclass longhorn -p '{"metadata": {"annotations":{"storageclass.kubernetes.io/is-default-class":"true"}}}'

### 8. Optional: Setup one or more [vclusters](https://www.vcluster.com/docs/getting-started/deployment)

In [ ]:
%%bash

kubectl create namespace vcluster-1
vcluster create vcluster-1 -n vcluster-1 --expose

In [ ]:
%%bash
vcluster connect vcluster-1 --namespace vcluster-1